# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 22:11:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P8     1W /  N/A |    371MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2020년 Celltrion Data**

In [5]:
lexicon_2020 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2020_celltrion.csv')

lexicon_2020.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,셀트리온,68270,매일경제,2020010210,"새해 문연 코스피, 개인 매수에 상승 출발 후 하락 전환",http://news.mk.co.kr/newsRead.php?no=3317&year...,\n\n\n 새해 첫 개장한 코스피가 개인의 매수에 힘입어 상승 출발한 후 하락 전...,2020-01-02,10,177742,179705,172832,176761,621826,-0.005525,-1,0,새해 개장 코스피 개인 매수 상승 출발 하락 전환 오전 현재 코스피 전일 대비 포인...


In [6]:
lexicon_2020[lexicon_2020['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
1331,셀트리온,68270,아시아경제,2020060215,"[속보] 당국 ""셀트리온 항체치료제, 유럽서 임상 준비중…혈장치료제 우선 확보 목표""",https://view.asiae.co.kr/article/2020060215162...,NaN,2020-06-03,15,227335,230772,220461,223407,1813508,0.013363,1,0,NaN


In [7]:
lexicon_2020.dropna(axis=0, inplace=True)
lexicon_2020 = lexicon_2020.reset_index(drop=True)
lexicon_2020.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2020 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2020['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2020['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2020['Tokenization'][x].split())):
                    if lexicon_2020['Tokenization'][x].split()[w] == list(set(lexicon_2020['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2020['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2020['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2020 = neg_lexicon_2020.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2020['date'][x], negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z])))

***2020-01-02 Cosine Similarity between <둔화> & <약세> : 0.5189449787139893
***2020-01-02 Cosine Similarity between <둔화> & <하락> : 0.5260781049728394
***2020-01-02 Cosine Similarity between <악화> & <둔화> : 0.5065981149673462
***2020-01-02 Cosine Similarity between <침체> & <경기> : 0.5360626578330994
***2020-01-02 Cosine Similarity between <침체> & <둔화> : 0.5701860189437866
***2020-01-02 Cosine Similarity between <하락> & <약세> : 0.5661999583244324
***2020-01-02 Cosine Similarity between <하락> & <상승> : 0.7126480340957642
***2020-01-02 Cosine Similarity between <하락> & <하락> : 0.9999999403953552
***2020-01-02 Cosine Similarity between <하락> & <보합> : 0.6176052689552307
***2020-01-02 Cosine Similarity between <하락> & <둔화> : 0.5260781049728394
***2020-01-02 Cosine Similarity between <둔화> & <약세> : 0.5189449787139893
***2020-01-02 Cosine Similarity between <둔화> & <하락> : 0.5260781049728394
***2020-01-02 Cosine Similarity between <악화> & <둔화> : 0.5065981149673462
***2020-01-02 Cosine Similarity between <침체> & <경기>

In [11]:
neg_lexicon_2020

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2020-01-02,1,둔화,약세,0.518945,2
1,2020-01-02,1,둔화,하락,0.526078,2
2,2020-01-02,1,악화,둔화,0.506598,3
3,2020-01-02,1,침체,경기,0.536063,1
4,2020-01-02,1,침체,둔화,0.570186,3
...,...,...,...,...,...,...
12535,2021-01-04,2028,둔화,증가,0.504186,2
12536,2021-01-04,2028,하락,반등,0.575454,4
12537,2021-01-04,2028,하락,상승,0.712648,1
12538,2021-01-04,2028,하락,보합,0.617605,1


In [12]:
neg_lexicon_2020.to_csv('../../../../Code/Data/Test/Stock-Year/neg_celltrion_2020.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2020['news_word'])))

a

['결함',
 '긍적적',
 '의심',
 '성공',
 '강점',
 '상승',
 '위축',
 '급등',
 '업황',
 '불안',
 '화물',
 '텍드림텍',
 '조치',
 '약화',
 '하락',
 '호전',
 '금수',
 '정체',
 '불황',
 '급락',
 '끝내',
 '둔화',
 '경색',
 '선박',
 '위기',
 '반등',
 '악화',
 '릴어드',
 '플러스',
 '강화',
 '폭락',
 '혼돈',
 '보합',
 '참패',
 '모상현',
 '증가',
 '좌절',
 '조처',
 '운항',
 '감소',
 '침체',
 '차량',
 '질책',
 '유발',
 '특혜',
 '부주의',
 '엠앤티',
 '저하',
 '절상',
 '급감',
 '보상',
 '등락',
 '논란',
 '약세',
 '호황',
 '폭등',
 '경기',
 '의혹',
 '주춤',
 '지연',
 '조아연',
 '견조',
 '금값',
 '부도',
 '스포츠투나잇',
 '병세',
 '악재']

In [14]:
b = list(set(list(neg_lexicon_2020[neg_lexicon_2020['cosine_similarity']>=0.7]['news_word'])))

b

['결함', '의심', '침체', '성공', '상승', '의혹', '부주의', '지연', '조치', '약화', '하락', '정체']

In [15]:
c = list(set(list(neg_lexicon_2020[neg_lexicon_2020['cosine_similarity']>=0.65]['news_word'])))

c

['결함',
 '긍적적',
 '의심',
 '침체',
 '성공',
 '상승',
 '의혹',
 '부주의',
 '지연',
 '조치',
 '약화',
 '하락',
 '정체',
 '급락',
 '조처']